# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}' #From testing sandbox 
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2217949917674,
 'week52high': 150.29,
 'week52low': 75.16,
 'week52highSplitAdjustOnly': 150.74,
 'week52lowSplitAdjustOnly': 75.26,
 'week52change': 0.7255671388893674,
 'sharesOutstanding': 16913876070,
 'float': 0,
 'avg10Volume': 110393149,
 'avg30Volume': 94196453,
 'day200MovingAvg': 128.56,
 'day50MovingAvg': 133.78,
 'employees': 153994,
 'ttmEPS': 4.49,
 'ttmDividendRate': 0.8463104772678455,
 'dividendYield': 0.006529344286124229,
 'nextDividendDate': '0',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-28',
 'peRatio': 29.306036491531426,
 'beta': 1.4046308243893848,
 'maxChangePercent': 50.7836647571242,
 'year5ChangePercent': 5.07550697398852,
 'year2ChangePercent': 1.54723603986093,
 'year1ChangePercent': 0.7439807906553197,
 'ytdChangePercent': -0.02092900278523583,
 'month6ChangePercent': 0.09528497063792857,
 'month3ChangePercent': -0.05315407883775376,
 'month1ChangePercent': 0.028054022579869706,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.7439807906553197

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data['AAPL']['stats'])
#     print(symbol_string.split(','))
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        
        index = my_columns),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,140.38,0.716452,N/A
1,AAL,22.34,1.36412,N/A
2,AAP,212.01,0.726992,N/A
3,AAPL,135.01,0.745054,N/A
4,ABBV,117.70,0.446897,N/A
...,...,...,...,...
500,YUM,126.76,0.487415,N/A
501,ZBH,181.22,0.516165,N/A
502,ZBRA,510.85,1.06705,N/A
503,ZION,59.08,1.10525,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,69.130,5,N/A
1,179,FCX,43.730,4.05141,N/A
2,208,GPS,35.580,4.03502,N/A
3,253,IVZ,29.755,2.96429,N/A
4,272,KSS,64.480,2.66396,N/A
5,441,TPR,49.580,2.42201,N/A
6,314,MOS,36.800,2.34167,N/A
7,410,SIVB,592.120,2.26868,N/A
8,129,DFS,122.510,2.10508,N/A
9,462,URI,360.660,2.09639,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_size():
    global portfolio_size
    
    
#     try:
#         float(portfolio_size)
#     except:
#         print('That is not a number! ')
#         print('Please try again:')
#         portfolio_size = input('Enter the size of your portfolio: ')
        
    while True:
        try:
            portfolio_size = float(input('Enter the size of your portfolio: '))
            break
        except ValueError:
            print("Enter a correct value!")
            pass

    #print '\nIncorrect input, try again'
        
portfolio_size()
print(portfolio_size)

final_dataframe

Enter the size of your portfolio: 
Enter a correct value!
Enter the size of your portfolio: 10000000
10000000.0


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,69.130,5,N/A
1,179,FCX,43.730,4.05141,N/A
2,208,GPS,35.580,4.03502,N/A
3,253,IVZ,29.755,2.96429,N/A
4,272,KSS,64.480,2.66396,N/A
5,441,TPR,49.580,2.42201,N/A
6,314,MOS,36.800,2.34167,N/A
7,410,SIVB,592.120,2.26868,N/A
8,129,DFS,122.510,2.10508,N/A
9,462,URI,360.660,2.09639,N/A


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

C:\Users\neran\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\neran\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number of shares to buy
0,275,LB,69.130,5,N/A,2893.0
1,179,FCX,43.730,4.05141,N/A,4573.0
2,208,GPS,35.580,4.03502,N/A,5621.0
3,253,IVZ,29.755,2.96429,N/A,6721.0
4,272,KSS,64.480,2.66396,N/A,3101.0
5,441,TPR,49.580,2.42201,N/A,4033.0
6,314,MOS,36.800,2.34167,N/A,5434.0
7,410,SIVB,592.120,2.26868,N/A,337.0
8,129,DFS,122.510,2.10508,N/A,1632.0
9,462,URI,360.660,2.09639,N/A,554.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A'             
            
            ],
        
            index = hqm_columns),
            ignore_index = True
        
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,138.830,N/A,0.726825,N/A,0.183555,N/A,0.0777989,N/A,0.01081,N/A
1,AAL,22.310,N/A,1.38734,N/A,0.916252,N/A,0.251198,N/A,-0.109208,N/A
2,AAP,211.350,N/A,0.715329,N/A,0.312849,N/A,0.300912,N/A,0.113158,N/A
3,AAPL,130.700,N/A,0.772538,N/A,0.097784,N/A,-0.0522941,N/A,0.0283202,N/A
4,ABBV,117.890,N/A,0.435697,N/A,0.293299,N/A,0.0827701,N/A,0.120341,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.370,N/A,0.498565,N/A,0.234835,N/A,0.161012,N/A,0.0646096,N/A
501,ZBH,182.729,N/A,0.521607,N/A,0.26468,N/A,0.087895,N/A,0.0615461,N/A
502,ZBRA,502.950,N/A,1.05707,N/A,0.366233,N/A,0.163824,N/A,-0.0414211,N/A
503,ZION,59.014,N/A,1.06614,N/A,0.900377,N/A,0.250358,N/A,0.0683242,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [20]:


time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])
# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe



TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: